In [3]:
from langchain.tools import tool
from tavily import TavilyClient

tavily_client = TavilyClient()  # uses TAVILY_API_KEY from env

@tool
def web_search(query: str) -> str:
    """Search the web (Tavily) for current movie info and return a concise summary."""
    result = tavily_client.search(
        query=query,
        search_depth="advanced",
        max_results=5,
        include_answer=True,
        include_raw_content=False,
    )

    # Tavily returns a dict; keep it readable for the LLM
    answer = result.get("answer", "")
    sources = result.get("results", [])

    lines = []
    if answer:
        lines.append(f"Answer: {answer}")
    lines.append("Top sources:")
    for s in sources[:5]:
        title = s.get("title", "Untitled")
        url = s.get("url", "")
        snippet = s.get("content", "")
        lines.append(f"- {title}\n  {url}\n  {snippet[:200]}")

    return "\n".join(lines)


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
SYSTEM_PROMPT = """
You are MovieScout, a helpful movie assistant.

Goals:
- Recommend movies based on the user's preferences (genre, mood, language, era, ratings, similar titles).
- If info might be time-sensitive (new releases, cast updates, streaming availability), use web_search.
- Ask 1 short clarifying question only if needed (e.g., "What mood?").

Rules:
- When you use web_search, cite the sources you used (title + URL).
- Keep answers structured: (1) quick recs list (2) short reasons (3) optional next question.
"""


In [7]:
from dotenv import load_dotenv
load_dotenv()

from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-nano",          # or "gpt-4o-mini" etc.
    tools=[web_search],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver()
)


In [8]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Suggest 5 movies like Interstellar.")]},
    config
)

print(response["messages"][-1].content)


Here are 5 movies similar to Interstellar:

- 2001: A Space Odyssey (1968) — epic, philosophical space exploration with grand ideas about humanity and technology.
- Gravity (2013) — intense space survival thriller with stunning realism and a tight, relentless pace.
- Ad Astra (2019) — a quiet, introspective space journey focused on family and personal discovery.
- The Martian (2015) — science-driven survival story with a practical, problem-solving vibe.
- Sunshine (2007) — a tense mission to reignite the sun, blending space adventure with philosophical questions.

Would you prefer more cerebral sci-fi or more action-packed space adventures?


In [9]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Suggest 5 movies like Interstellar.', additional_kwargs={}, response_metadata={}, id='9b7913c1-88d2-4080-bc47-6ebc9c899c00'),
              AIMessage(content='Here are 5 movies similar to Interstellar:\n\n- 2001: A Space Odyssey (1968) — epic, philosophical space exploration with grand ideas about humanity and technology.\n- Gravity (2013) — intense space survival thriller with stunning realism and a tight, relentless pace.\n- Ad Astra (2019) — a quiet, introspective space journey focused on family and personal discovery.\n- The Martian (2015) — science-driven survival story with a practical, problem-solving vibe.\n- Sunshine (2007) — a tense mission to reignite the sun, blending space adventure with philosophical questions.\n\nWould you prefer more cerebral sci-fi or more action-packed space adventures?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1490, 'prompt_tokens': 265, 'total_tokens': 1755, 'co